In [61]:
from IPython.display import Markdown
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [62]:
# Import data
df = pd.read_pickle(r'data\04_fct\fct_demographic_offers_and_transactions.pkl')
df.head()

,age,income,days_as_member,gender_F,gender_M,offer_viewed,offer_completed,viewed_before_completion,difficulty,reward,duration_hrs,mobile,social,web,is_bogo,is_discount,total_transactions,total_transaction_amount
customer_id,,,,,,,,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,0,1,0,5,5,168,1,0,1,1,0,1,23.22
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,0,0,0,0,0,96,1,0,1,0,0,0,0.00
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,1,1,1,5,5,168,1,0,1,1,0,1,19.89
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,1,0,0,0,0,72,1,1,0,0,0,0,0.00
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,1,1,1,10,10,168,1,1,0,1,0,1,21.72


In [63]:
def process_data(data, gender_column):
    # Filter data for the specified gender
    gender_data = data[data[gender_column] == 1]
    # Drop 'gender_F' and 'gender_M' columns
    gender_data = gender_data.drop(['gender_F', 'gender_M'], axis=1)
    # Extract features and target variable
    features = gender_data[['mobile', 'social', 'web']]
    target = gender_data['offer_viewed']
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

def train_evaluate_model(X_train, X_test, y_train, y_test):
    # Initialize and train the random forest classifier with balanced class weights
    rf_clf = RandomForestClassifier(random_state=42, class_weight='balanced')
    rf_clf.fit(X_train, y_train)
    # Make predictions
    y_pred = rf_clf.predict(X_test)
    y_prob = rf_clf.predict_proba(X_test)[:, 1]
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_prob)
    # Calculate permutation importance
    perm_importance = permutation_importance(rf_clf, X_test, y_test, n_repeats=10, random_state=42)
    perm_importance_df = pd.DataFrame({'Feature': ['mobile', 'social', 'web'],
                                       'Importance': perm_importance.importances_mean,
                                       'Importance STD': perm_importance.importances_std})
    return {
        "Accuracy": accuracy,
        "Confusion Matrix": conf_matrix,
        "Classification Report": class_report,
        "ROC-AUC Score": roc_auc,
        "Permutation Importance": perm_importance_df
    }

# Process data for female and male customers
X_train_female, X_test_female, y_train_female, y_test_female = process_data(df, 'gender_F')
X_train_male, X_test_male, y_train_male, y_test_male = process_data(df, 'gender_M')

# Train and evaluate models for female and male customers
results_female = train_evaluate_model(X_train_female, X_test_female, y_train_female, y_test_female)
results_male = train_evaluate_model(X_train_male, X_test_male, y_train_male, y_test_male)

In [64]:
### Parse model's evaluation results ###
# Initialize the data dictionary
data = {
    "Metric": [],
    "Female": [],
    "Male": []
}

data["Metric"].append("ROC-AUC Score")
data["Female"].append(results_female["ROC-AUC Score"])
data["Male"].append(results_male["ROC-AUC Score"])

# Calculate FPR for Female
fp_female = results_female["Confusion Matrix"].flatten()[1]  # FP is at index 1
tn_female = results_female["Confusion Matrix"].flatten()[0]  # TN is at index 0
fpr_female = fp_female / (fp_female + tn_female) if (fp_female + tn_female) > 0 else 0

# Calculate FPR for Male
fp_male = results_male["Confusion Matrix"].flatten()[1]  # FP is at index 1
tn_male = results_male["Confusion Matrix"].flatten()[0]  # TN is at index 0
fpr_male = fp_male / (fp_male + tn_male) if (fp_male + tn_male) > 0 else 0

# Add FPR to the data dictionary
data["Metric"].append("False Positive Rate")
data["Female"].append(fpr_female)
data["Male"].append(fpr_male)

# Calculate FNR for Female
fn_female = results_female["Confusion Matrix"].flatten()[2]  # FN is at index 2
tp_female = results_female["Confusion Matrix"].flatten()[3]  # TP is at index 3
fnr_female = fn_female / (fn_female + tp_female) if (fn_female + tp_female) > 0 else 0

# Calculate FNR for Male
fn_male = results_male["Confusion Matrix"].flatten()[2]  # FN is at index 2
tp_male = results_male["Confusion Matrix"].flatten()[3]  # TP is at index 3
fnr_male = fn_male / (fn_male + tp_male) if (fn_male + tp_male) > 0 else 0

# Add FNR to the data dictionary
data["Metric"].append("False Negative Rate")
data["Female"].append(fnr_female)
data["Male"].append(fnr_male)

# Function to extract F1-scores for macro and weighted averages
def extract_f1_scores(report):
    lines = report.split('\n')
    f1_scores = {}
    for line in lines:
        if "macro avg" in line:
            f1_scores['F1-Score (Macro)'] = line.split()[-2]  # Assuming F1-score is the second last element
        elif "weighted avg" in line:
            f1_scores['F1-Score (Weighted)'] = line.split()[-2]  # Assuming F1-score is the second last element
    return f1_scores

# Extract F1-scores for macro and weighted averages
f1_scores_female = extract_f1_scores(results_female['Classification Report'])
f1_scores_male = extract_f1_scores(results_male['Classification Report'])

# Assuming classification_metrics includes the metrics you're interested in
classification_metrics = ['F1-Score (Macro)', 'F1-Score (Weighted)']

# Loop through each metric and append the results to the data dictionary
for metric_name in classification_metrics:
    data["Metric"].append(metric_name)
    data["Female"].append(f1_scores_female[metric_name])
    data["Male"].append(f1_scores_male[metric_name])

# Add permutation importance
for index, feature in enumerate(results_female['Permutation Importance']['Feature']):
    data["Metric"].append(f'Importance of {feature}')
    data["Female"].append(results_female['Permutation Importance']['Importance'][index])
    data["Male"].append(results_male['Permutation Importance']['Importance'][index])
    
    data["Metric"].append(f'Importance STD of {feature}')
    data["Female"].append(results_female['Permutation Importance']['Importance STD'][index])
    data["Male"].append(results_male['Permutation Importance']['Importance STD'][index])

# Create the DataFrame
df = pd.DataFrame(data)

# Calculate the percent difference
# Convert 'Female' and 'Male' columns to numeric (float) to ensure calculations can be performed
df['Female'] = pd.to_numeric(df['Female'], errors='coerce')
df['Male'] = pd.to_numeric(df['Male'], errors='coerce')

# Calculate percent difference
condition_both_non_zero = (df['Female'] != 0) & (df['Male'] != 0)
condition_one_zero = (df['Female'] == 0) | (df['Male'] == 0)
condition_both_zero = (df['Female'] == 0) & (df['Male'] == 0)

# Calculate percent difference
df['Percent Difference'] = np.nan
df.loc[condition_both_non_zero, 'Percent Difference'] = ((df['Female'] - df['Male']) / ((df['Female'] + df['Male']) / 2)) * 100
df.loc[condition_one_zero & ~condition_both_zero, 'Percent Difference'] = np.nan

df.to_csv(r'data/04_fct/fct_offer_channel_importance_evaluation_results.csv', index=False)
df

,Metric,Female,Male,Percent Difference
0,ROC-AUC Score,0.811208,0.840370,-3.531331
1,False Positive Rate,0.175746,0.125984,32.984143
2,False Negative Rate,0.285041,0.248058,13.874682
3,F1-Score (Macro),0.700000,0.750000,-6.896552
4,F1-Score (Weighted),0.760000,0.800000,-5.128205
5,Importance of mobile,0.014891,0.000000,NaN
6,Importance STD of mobile,0.001376,0.000000,NaN
7,Importance of social,0.189478,0.229724,-19.201104
8,Importance STD of social,0.003960,0.002732,36.691376
9,Importance of web,0.000000,0.000000,NaN
